# Simple Neurons: The Basics Of Neural Networks
This notebook is where I explore neural networks on a much more basic and fundamental level. Instead of focusing on creating a NN that can solve the universe, I want to fully aquaint myself with the fundamentals of linear algebra in ML, backpropogation and optimisation techniques. That's what this notebook is aimed to do: simply learn the basics in a much more controlled way.

I'll be using a simple case of three input neurons, _no_ hidden neurons and a single output neuron. I've designed the system this way since I want to attempt to "learn" the behaviour of an [AND](https://en.wikipedia.org/wiki/AND_gate) gate and an [OR](https://en.wikipedia.org/wiki/OR_gate) gate.

## Acknowledgements
Once again, I'm using a plethora of material to learn about NN and how to implement them. In no particular order, here are my main reads that have helped me massively.
 - iamtrask's blog article on "[A Neural Network in 11 lines of Python](https://iamtrask.github.io/2015/07/12/basic-python-network/)"
 - Matt Mazur's fantastic article on the theory behind backpropogation, "[A Step by Step Backpropagation Example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)" 
 - Analytics Vidhya's article on "[Understanding and coding Neural Networks From Scratch in Python and R](https://www.analyticsvidhya.com/blog/2017/05/neural-network-from-scratch-in-python-and-r/)"

# Developing the Neural Network
Before we can actually do anything, we need to develop the neural network that we will be using. Fortunately I already (roughly) know how to do that.

In [1]:
# Import the important packages.
import pandas as pd
import numpy as np

# Define any globals.
np.random.seed(0)
DATA_POINTS = 10000  # 10 Thousand
INPUT_NEURONS = 3

In [2]:
# Create the important functions
def sigmoid(x):
    """Normalises the inputs to be between 0 and 1"""
    return 1.0/(1.0+np.exp(-x))

def S(x):
    """Shorthand notation for the sigmoid function"""
    return sigmoid(x)

def dS(x_norm):
    """Shorthand notation for the derivative of the sigmoid function"""
    return x_norm*(1-x_norm)

def errorFunc(expected, actual):
    """Calculates the error"""
    return 0.5*(expected - actual)**2

In [3]:
# Create the NN class.
class NeuralNetwork(object):
    """The neural network."""
    
    def __init__(self, x, y, seed=None):
        """Initialise the NN"""        
        self.x = x
        self.y = y
        
        if seed: np.random.seed(0)
        
        self.weights = 2*np.random.random((3,1)) - 1
        self.bias = 2*np.random.random((1, 1)) - 1
        return None
    
    def train(self, epochs=1, verbose=False):
        """Trains the network for the given iterations"""        
        for i in xrange(epochs):  # Go over the data N times
            if verbose: print "%i/%i epochs complete" % (i, epochs)
            for x, y in zip(self.x, self.y):  # The data itself (to avoid matrix mechanics)
                # Forward propagation
                output = S(np.dot(x.T, self.weights) + self.bias[0])

                # Calculate error
                error = errorFunc(y, output)

                # Backward propagate weights
                learning_rate = error
                delta = dS(output)*(output-y)
                self.weights = self.weights + np.array([(-delta*learning_rate)[0]*x]).T
                self.bias = self.bias - [delta*learning_rate]  # Since all bias "inputs" are 1
        
        if verbose:
            print "Weights:", list(self.weights)
            print "Bias:", list(self.bias)
        return None
    
    def think(self, x):
        """Given the inputs x, let the NN predict y."""
        # Forward propagation
        input_layer = x
        output_layer = S(np.dot(input_layer, self.weights) + self.bias[0])
        return output_layer[0]

# Part 1 - The AND gate
The first problem I'll be attempting to solve is for the AND gate. This gate takes boolean inputs (True or False) and outputs True if all inputs are True or False if any inputs are False. To express this in Boolean algebra:

> AND(A, B, C) = ABC

where the inputs can only be 0 or 1. Intuatively I expect that, without normalisation, the weights $w_1$, $w_2$ and $w_3$ will all be 0.33 and the bias, $b$, will be -0.16. That's because the weighted summation will then only be above 0.5 (the "True" threshold) when all three inputs fire.

With this in mind, let's now generate the data for the neural network.

In [4]:
def AND(x, y, z):
    return x and y and z

In [5]:
x1 = np.random.randint(2, size=(INPUT_NEURONS*DATA_POINTS))
x1 = np.split(x1, DATA_POINTS)

y1 = [AND(a, b, c) for a, b, c in x1]

With the data fully primed and ready, we can now train the neural network and see how good of a result it gets.

In [6]:
NN = NeuralNetwork(x1, y1, seed=1)
NN.train(200)

To confirm that the model works, we can test it against the various inputs of the AND gate to see what it thinks the answer might be.

In [7]:
validation_x = [
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1],
]

validation_y = [AND(a, b, c) for a, b, c in validation_x]

# Now see how accurate it is.
count = 0
print "ACCURACY CHECK"
print "=================="
for x, y in zip(validation_x, validation_y):
    prediction = NN.think(x)
    print "%s: %s -> %i" % (x, prediction, int(round(prediction)))
    count += int(int(round(prediction) == y))
print "\nAccuracy:", float(count)/len(validation_y)

ACCURACY CHECK
[0, 0, 0]: 2.41194618326e-06 -> 0
[0, 0, 1]: 0.000405165926404 -> 0
[0, 1, 0]: 0.000403058006319 -> 0
[0, 1, 1]: 0.0634612518911 -> 0
[1, 0, 0]: 0.000412239702661 -> 0
[1, 0, 1]: 0.0648137640716 -> 0
[1, 1, 0]: 0.0644981850056 -> 0
[1, 1, 1]: 0.920548253539 -> 1

Accuracy: 1.0


# Part 2 - The OR Gate
With the AND gate all done and dusted, we can shift our attention to getting the OR gate data to play around with. The OR gate returns True if any of the inputs are True. You can implement it in Boolean algebra like so:

> OR(A, B, C) = A + B + C

In this case I expect that the bias should start negative, but the weights on the input neurons will be orders of magnitude greater than the bias's. We can test that theory out quickly!

In [8]:
def OR(x, y, z):
    return x or y or z

In [9]:
x2 = np.random.randint(2, size=(INPUT_NEURONS*DATA_POINTS))
x2 = np.split(x2, DATA_POINTS)

y2 = [OR(a, b, c) for a, b, c in x2]

In [10]:
NN = NeuralNetwork(x2, y2, seed=1)
NN.train(200)

In [11]:
validation_x = [
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1],
]

validation_y = [OR(a, b, c) for a, b, c in validation_x]

# Now see how accurate it is.
count = 0
print "ACCURACY CHECK"
print "=================="
for x, y in zip(validation_x, validation_y):
    prediction = NN.think(x)
    print "%s: %s -> %i" % (x, prediction, int(round(prediction)))
    count += int(int(round(prediction) == y))
print "\nAccuracy:", float(count)/len(validation_y)

ACCURACY CHECK
[0, 0, 0]: 0.0639726575546 -> 0
[0, 0, 1]: 0.953637848106 -> 1
[0, 1, 0]: 0.953437384377 -> 1
[0, 1, 1]: 0.999837758972 -> 1
[1, 0, 0]: 0.953964244169 -> 1
[1, 0, 1]: 0.999839683023 -> 1
[1, 1, 0]: 0.999838956098 -> 1
[1, 1, 1]: 0.99999946482 -> 1

Accuracy: 1.0


# Conclusion
As you can see, a __very__ simple neural network can be implemented in a very simple manner and you can teach it to understand AND and OR gates with minimal effort. This however is exactly that: simple. It can't predict what image it is looking at, nor can it predict more complex gates like XOR. That might take me a while longer to understand and be able to write. 

Hopefully my NN has been somewhat useful and interesting to read. Please send me a message if you have any improvements to this notebook or y